In [112]:
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp
import face_recognition_models
import dlib
import os
from mediapipe.python.solutions.drawing_utils import _normalized_to_pixel_coordinates
from time import time
import matplotlib.pyplot as plt
import imutils
from imutils import face_utils
import face_recognition
import shutil

In [113]:
face_recognition_model = face_recognition_models.face_recognition_model_location()
face_encoder = dlib.face_recognition_model_v1(face_recognition_model)

In [114]:
landmark_points_68 = [162,234,93,58,172,136,149,148,152,377,378,365,397,288,323,454,389,71,63,105,66,107,336,
                  296,334,293,301,168,197,5,4,75,97,2,326,305,33,160,158,133,153,144,362,385,387,263,373,
                  380,61,39,37,0,267,269,291,405,314,17,84,181,78,82,13,312,308,317,14,87]
    
landmark_points_5_1 = [ 2, #bottom of nose tip
                     362, #left eye towards centre
                     263, #left eye away from centre
                     33,  #right eye away from centre
                     133 #right eye towards centre 
                    ]
landmark_points_5_2 = [ 2, #bottom of nose tip
                     263, #left eye away from centre
                     362, #left eye towards centre
                     133, #right eye towards centre 
                     33  #right eye away from centre
                    ]


In [115]:
mp_drawing = mp.solutions.drawing_utils

mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=1,min_detection_confidence=0.7)

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, refine_landmarks=False,max_num_faces=1,min_detection_confidence=0.5)

### MP + Dlib utils


In [116]:
def landmark_pt_list(mesh_results,width,height):
    landmark_points_68 = [162,234,93,58,172,136,149,148,152,377,378,365,397,288,323,454,389,71,63,105,66,107,336,
                  296,334,293,301,168,197,5,4,75,97,2,326,305,33,160,158,133,153,144,362,385,387,263,373,
                  380,61,39,37,0,267,269,291,405,314,17,84,181,78,82,13,312,308,317,14,87]
    
    landmark_points_5_1 = [ 2, #bottom of nose tip
                     362, #left eye towards centre
                     263, #left eye away from centre
                     33,  #right eye away from centre
                     133 #right eye towards centre 
                    ]
    landmark_points_5_2 = [ 2, #bottom of nose tip
                     263, #left eye away from centre
                     362, #left eye towards centre
                     133, #right eye towards centre 
                     33  #right eye away from centre
                    ]
    landmark_points_5 = [ 263, #left eye away from centre
                       362, #left eye towards centre
                       33,  #right eye away from centre
                       133, #right eye towards centre
                        2 #bottom of nose tip 
                    ]

    if mesh_results.multi_face_landmarks:
#         print(mesh_results.multi_face_landmarks)
        for i,face_landmarks in enumerate(mesh_results.multi_face_landmarks): 
            if i==0:
                raw_landmark_set = []
                for index in landmark_points_5:
                    x = int(face_landmarks.landmark[index].x * width)
                    y = int(face_landmarks.landmark[index].y * height)
                    landmark_point=dlib.point([x,y])
                    raw_landmark_set.append(landmark_point)
                    #display(landmark_point)
                all_points=dlib.points(raw_landmark_set)
#         return dlib.points([{
#             "nose_tip": [raw_landmark_set[0]],
#             "left_eye": raw_landmark_set[1:3],
#             "right_eye": raw_landmark_set[3:],
#         }])
        return all_points

def bounding_rect(detection_results,width,height):
    if detection_results.detections:
        for i,detection in enumerate(detection_results.detections):
            if i==0:
                # bbox data
                bbox = detection.location_data.relative_bounding_box
                xy_min = _normalized_to_pixel_coordinates(bbox.xmin, bbox.ymin, width,height)
                xy_max = _normalized_to_pixel_coordinates(bbox.xmin + bbox.width, bbox.ymin + bbox.height,width,height)
                if xy_min is None or xy_max is None:
                    #print("face out of frame")
                    return
                else:
                    left,top =xy_min
                    right,bottom = xy_max
                    #bbox_points = { "xmin" : xmin,"ymin" : ymin,"xmax" : xmax,"ymax" : ymax}
                    rectangle= dlib.rectangle(left=left, bottom=bottom, right=right, top=top)
                    return rectangle

def ret_encoding(filepath,num_jitters=1):
    image_input = cv2.imread(filepath)
#     print("face test")
#     print(image_input.shape)
#     image_input =face_recognition.load_image_file(filepath)
#     image = cv2.imread(cap_path)        

    image_input = cv2.cvtColor(image_input, cv2.COLOR_BGR2RGB)
#     print(image_input.shape)

#     width,height=image_input.shape[:-1]
    height, width, _ = image_input.shape
    detection_results = face_detection.process(image_input)
    mesh_results = face_mesh.process(image_input)
    
    all_points=  landmark_pt_list(mesh_results,width,height)  
#     print("all_points", all_points)

    b_box=bounding_rect(detection_results,width,height)
#     print(b_box)
#     print(b_box.left(),b_box.right(),b_box.top(),b_box.bottom())

    if (all_points is None) or (b_box is None):
        return 
    raw_landmark_set=dlib.full_object_detection(b_box,all_points)
    #display(all_points)
    #display(b_box)
#     cv2.imshow("test", image_input)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    encodings=face_encoder.compute_face_descriptor(image_input, raw_landmark_set, num_jitters)

    return encodings



In [117]:
# ret_encoding(filepath="/Users/michaelmandiberg/Documents/projects-active/facemap_production/just_start_image/portrait-of-funny-afro-guy-picture-id1402424532.jpg")

### Linear sorting

In [118]:
def get_img_list(folder):
    img_list=[]
    for file in os.listdir(folder):
        if not file.startswith('.') and os.path.isfile(os.path.join(folder, file)):
            filepath = os.path.join(folder, file)
            filepath=filepath.replace('\\' , '/')
            img_list.append(file)
    return img_list        
    print("got image list")

In [119]:
def get_d(enc1, enc2):
    enc1=np.array(enc1)
    enc2=np.array(enc2)
    d=np.linalg.norm(enc1 - enc2, axis=0)
    return d

In [120]:
# not df
def encode_list(folder, img_list):
    enc_dict={}
    for img in img_list:
        img_path = os.path.join(folder,img)
        encoding=(ret_encoding(img_path))
        enc_dict[img]=encoding 
    return enc_dict

In [121]:
# not df
def get_closest(folder, start_img, img_list, enc_dict):
    img_list.remove(start_img)
#     enc1=(ret_encoding(os.path.join(folder,start_img)))
    enc1=enc_dict[start_img]
    dist=[]
    dist_dict={}
    for img in img_list:
#         file2=filebase+str(i)+".jpg".replace('\\' , '/')
#         test_img = os.path.join(folder,img)
#         enc2=(ret_encoding(test_img))
        enc2 = enc_dict[img]
        if (enc1 is not None) and (enc2 is not None):
#             print(file2)
            d = get_d(enc1, enc2)
#             print(d)
            dist.append(d)
            dist_dict[d]=img
    dist.sort()
#     print(len(dist))
    return dist[0], dist_dict[dist[0]]

In [122]:
def save_sorted(folder, image, counter, dist):
    sorted_name = "linear_sort_"+str(counter)+"_"+str(round(dist, 2))+".jpg"
    sortfolder="sorted2"
    newfolder = os.path.join(folder,sortfolder)
    old_name=os.path.join(folder,image)
    new_name=os.path.join(newfolder,sorted_name)
    if not os.path.exists(newfolder):
        os.makedirs(newfolder)
    shutil.copy(old_name, new_name)
    print('saved, ',sorted_name)


### dataframe creation and sorting

In [123]:
def encode_list_df(folder, img_list):
#     enc_dict={}
    csv_name="face_encodings.csv"
    col1="file_name"
    col2="encoding"
    curr=0
    total = len(img_list)

    # encodings column list for splitting
    col_list=[]
    for i in range(128):
        col_list.append(col2+str(i))

    #initializing the dataframe
    image_data=pd.DataFrame(columns=[col1, col2])

    
    for img in img_list:
        if curr%10==0:print(curr,"/",total)
        curr+=1
        filepath = os.path.join(folder,img)        
        filepath=filepath.replace('\\' , '/')  ## cv2 accepts files with "/" instead of "\"
        encodings=ret_encoding(filepath)
#         print(encodings)
        if encodings is not None:              ## checking if a face is found
            data=pd.DataFrame({col1:img,col2:[np.array(encodings)]})
            image_data = pd.concat([image_data,data],ignore_index=True)  

    #splitting the encodings column
    output_data = pd.DataFrame(image_data[col2].to_list(), columns=col_list)
    #adding the filename column and then puting it first
    output_data[[col1]]=pd.DataFrame(image_data[col1].tolist(),index=image_data.index)
    clms = output_data.columns.tolist()
    clms = clms[-1:] + clms[:-1]
    output_data=output_data[clms]
    # saving without index
    output_data.to_csv(csv_name, index=False)
    df = pd.read_csv(csv_name)
    return output_data

In [124]:
def get_closest_df(folder, start_img, df_enc):
    if start_img == "median":
        enc1 = df_enc.median().to_list()
        print("in median")
        print(enc1)
    elif start_img == "start_site_image_id":
        print("start_site_image_id")
        print(start_site_image_id)
        enc1 = df_enc.loc[start_site_image_id].to_list()
        print(enc1)
        
    else:
#         enc1 = get 2-129 from df via stimg key
        print("start_img key is:")
        print(start_img)
        enc1 = df_enc.loc[start_img].to_list()
        df_enc=df_enc.drop(start_img)
        print("in new img",len(df_enc.index))
        print(enc1)
    
#     img_list.remove(start_img)
#     enc1=enc_dict[start_img]
    
    dist=[]
    dist_dict={}
    for index, row in df_enc.iterrows():
#         print(row['c1'], row['c2'])
#     for img in img_list:
        enc2 = row
        if (enc1 is not None) and (enc2 is not None):
            d = get_d(enc1, enc2)
            dist.append(d)
            dist_dict[d]=index
    dist.sort()
#     print(len(dist))
    return dist[0], dist_dict[dist[0]], df_enc

In [125]:
# test if new and old make a face
def is_face(image):
    # For static images:
    # I think this list is not used
    IMAGE_FILES = []
    with mp_face_detection.FaceDetection(model_selection=1, 
                                        min_detection_confidence=0.6
                                        ) as face_detection:
        # image = cv2.imread(file)
        # Convert the BGR image to RGB and process it with MediaPipe Face Detection.
#         detection_results = face_detection.process(image)

        results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # Draw face detections of each face.
        if not results.detections:
            is_face = False
        else:
            is_face = True
        # annotated_image = image.copy()
        # for detection in results.detections:
        #     is_face = True
        #     print('Nose tip:')
        #     print(mp_face_detection.get_key_point(
        #       detection, mp_face_detection.FaceKeyPoint.NOSE_TIP))
        #     mp_drawing.draw_detection(annotated_image, detection)
        # cv2.imwrite('/tmp/annotated_image' + str(idx) + '.png', annotated_image)

        return is_face


In [126]:
# test if new and old make a face
def test_pair(last_file, new_file):
    try:
        img = cv2.imread(new_file)
        height, width, layers = img.shape
        size = (width, height)
        print('loaded img 1')
        
        last_img = cv2.imread(new_file)
        last_height, last_width, last_layers = last_img.shape
        last_size = (last_width, last_height)
        print('loaded img 2')
        
        # test to see if this is actually an face, to get rid of blank ones/bad ones
        if is_face(img):
            print('new file is face')
            # if not the first image
#             if i>0:
            # blend this image with the last image
            blend = cv2.addWeighted(img, 0.5, last_img, 0.5, 0.0)
            print('blended faces')
            blended_face = is_face(blend)
            print('is_face ',blended_face)
            # if blended image has a detectable face, append the img
            if blended_face:
#                     img_array.append(img)
                print('is a face! adding it')
                return True
            else:
                print('skipping this one')
                return False
            # for the first one, just add the image
            # this may need to be refactored in case the first one is bad?
#             else:
#                 print('this is maybe the first round?')
#                 img_array.append(img)
        else:
            print('new_file is not face: ',new_file)
            return False

#         i+=1

    except:
        print('failed:',new_file)
        return False


In [132]:
folder="/Users/michaelmandiberg/Documents/projects-active/facemap_production/images1684545820.98789"


img_list = get_img_list(folder)

# start_img = img_list[1]
start_img = "median"

# start_img = "start_site_image_id"
# start_site_image_id = "faceimg_crop1_X-15toX5_Y-4toY4_Z-3toZ3_maxResize0.3_ct2412_1_1402424532.jpg"

# faceimg_crop1_X-15toX5_Y-4toY4_Z-3toZ3_maxResize0.3_ct2412_18_1396511988.jpg

# enc_dict = encode_list(folder, img_list)

df_enc = encode_list_df(folder, img_list)
print(df_enc)

df_enc.set_index('file_name', inplace=True)


#with lists/dicts
# dist=0
# for i in range(len(img_list)-1):
#     save_sorted(folder, start_img, i, dist)
#     dist, start_img = get_closest(folder, start_img,img_list, enc_dict)
    
#     print(dist)
#     print (start_img)
    
#     if dist > .37: 
#         continue

#with df
dist=0
print(len(df_enc.index))


    
for i in range(len(df_enc.index)-2):
    dist, start_img, df_enc = get_closest_df(folder, start_img,df_enc)
#     print(folder, start_img, i, dist)
    save_sorted(folder, start_img, i, dist)
#     print(i)
#     print(len(df_enc.index))
#     print(dist)
#     print (start_img)
    
    if dist > .60: 
        break
    
    
# I don't know why, but this isn't working
# for i in range(len(df_enc.index)-2):
#     dist, start_img, df_enc = get_closest_df(folder, start_img,df_enc)
# #     print(folder, start_img, i, dist)
#     save_sorted(folder, start_img, i, dist)

#     if i>0:
#         #test blend
# #         last_file = os.path.join(folder,)
#         blend_is_face = (test_pair(os.path.join(folder,last_img), os.path.join(folder,start_img)))
#         print('blend_is_face ',blend_is_face)
#         if blend_is_face:
# #         print(test_pair(last_img,start_img))
#             save_sorted(folder, start_img, i, dist)
#             last_img = start_img
# #         else:
# #             start_img = last_img
            
#     print(i)
#     print(len(df_enc.index))
#     print(dist)
#     print (start_img)
    
#     if dist > .37: 
#         break
        
       

    

0 / 337
10 / 337
20 / 337
30 / 337
40 / 337
50 / 337
60 / 337
70 / 337
80 / 337
90 / 337
100 / 337
110 / 337
120 / 337
130 / 337
140 / 337
150 / 337
160 / 337
170 / 337
180 / 337
190 / 337
200 / 337
210 / 337
220 / 337
230 / 337
240 / 337
250 / 337
260 / 337
270 / 337
280 / 337
290 / 337
300 / 337
310 / 337
320 / 337
330 / 337
                                             file_name  encoding0  encoding1  \
0    faceimg_crop1_X-15toX5_Y-4toY4_Z-3toZ3_maxResi...  -0.225971   0.067224   
1    faceimg_crop1_X-15toX5_Y-4toY4_Z-3toZ3_maxResi...  -0.109089  -0.005886   
2    faceimg_crop1_X-15toX5_Y-4toY4_Z-3toZ3_maxResi...  -0.132457   0.058646   
3    faceimg_crop1_X-15toX5_Y-4toY4_Z-3toZ3_maxResi...  -0.112790   0.041324   
4    faceimg_crop1_X-15toX5_Y-4toY4_Z-3toZ3_maxResi...  -0.118518   0.103365   
..                                                 ...        ...        ...   
331  faceimg_crop1_X-15toX5_Y-4toY4_Z-3toZ3_maxResi...  -0.145695   0.043081   
332  faceimg_crop1_X-15toX5_Y-4

saved,  linear_sort_12_0.55.jpg
start_img key is:
faceimg_crop1_X-15toX5_Y-4toY4_Z-3toZ3_maxResize0.3_ct2412_181_1136450911.jpg
in new img 323
[-0.11586350202560425, 0.06880972534418106, 0.04344464838504791, -0.10124813765287399, -0.16163791716098785, 0.05425349622964859, -0.04219112917780876, -0.04322747141122818, 0.21886304020881653, -0.1317043900489807, 0.23990027606487274, 0.01879735291004181, -0.218246191740036, -0.0005377680063247681, 0.0372125506401062, 0.14256907999515533, -0.2099788784980774, -0.09188229590654373, -0.07805215567350388, -0.03548334538936615, 0.01345556229352951, 0.0662156343460083, -0.06627030670642853, 0.05350695922970772, -0.12778913974761963, -0.40147101879119873, -0.08559196442365646, -0.1188935935497284, -0.025933079421520233, -0.1415005773305893, 0.03380868583917618, 0.11621555685997009, -0.1577490270137787, -0.032495781779289246, -0.029525727033615112, 0.039990171790122986, -0.09323882311582565, -0.11177228391170502, 0.21371793746948242, 0.05108761042356

saved,  linear_sort_27_0.36.jpg
start_img key is:
faceimg_crop1_X-15toX5_Y-4toY4_Z-3toZ3_maxResize0.3_ct2412_100_900602970.jpg
in new img 308
[-0.0331602469086647, 0.024709612131118774, -0.012242268770933151, -0.03853572905063629, -0.11422523856163025, -0.07524341344833374, -0.06878107041120529, -0.12084642052650452, 0.11900445073843002, -0.08930482715368271, 0.2289690524339676, -0.045559778809547424, -0.18625515699386597, -0.1282925009727478, -0.04115907847881317, 0.20270279049873352, -0.21560603380203247, -0.12758231163024902, -0.048622872680425644, 0.017726436257362366, 0.07969699054956436, 0.013229367323219776, 0.02759653702378273, 0.05721767246723175, -0.12728926539421082, -0.3409491777420044, -0.08494555950164795, -0.11295855790376663, -0.027473289519548416, -0.06544803082942963, -0.06980752944946289, 0.06574849784374237, -0.1524626612663269, -0.07819031924009323, 0.06097779422998428, 0.05961785465478897, -0.011271253228187561, -0.07543937116861343, 0.17193251848220825, 0.0204449

saved,  linear_sort_43_0.52.jpg
start_img key is:
faceimg_crop1_X-15toX5_Y-4toY4_Z-3toZ3_maxResize0.3_ct2412_133_1152107462.jpg
in new img 292
[-0.08175501227378845, 0.13771064579486847, -0.00980463344603777, -0.025843529030680656, -0.0779695212841034, -0.03746386244893074, -0.0031156055629253387, -0.1548282653093338, 0.14164917171001434, -0.01604580506682396, 0.26530131697654724, -0.04937169700860977, -0.21320118010044098, -0.14264406263828278, -0.04374352842569351, 0.11863537132740021, -0.20482879877090454, -0.1078944206237793, -0.01652468368411064, 0.01486368477344513, 0.17490971088409424, 0.0041708387434482574, 0.05827382951974869, 0.07338092476129532, -0.10461784899234772, -0.3118358850479126, -0.048744816333055496, -0.11399313807487488, 0.10451574623584747, -0.034516990184783936, -0.08924325555562973, -0.02999657392501831, -0.235458642244339, -0.12351975589990616, 0.02369990199804306, 0.04307880997657776, -0.009858660399913788, -0.08044836670160294, 0.18751224875450134, -0.015504

saved,  linear_sort_58_0.44.jpg
start_img key is:
faceimg_crop1_X-15toX5_Y-4toY4_Z-3toZ3_maxResize0.3_ct2412_87_1366508246.jpg
in new img 277
[-0.0825086236000061, -0.049469396471977234, -0.020511142909526825, -0.0752033144235611, -0.1383284330368042, -0.04846300929784775, -0.02594681829214096, -0.08492454141378403, 0.13940638303756714, -0.15350906550884247, 0.2078600823879242, -0.09523452073335648, -0.23932406306266785, -0.04716198146343231, -0.09085599333047867, 0.20217101275920868, -0.16491135954856873, -0.12347091734409332, -0.03522711992263794, -0.02384638786315918, 0.08740626275539398, 0.015000615268945694, -0.05800637975335121, 0.045911967754364014, -0.12538012862205505, -0.2900908589363098, -0.10819883644580841, -0.04812224954366684, -0.03970485180616379, -0.0553743839263916, -0.014930512756109238, 0.0949348732829094, -0.1511600911617279, -0.04422663524746895, 0.05366520583629608, 0.11513403058052063, -0.03402375429868698, -0.09744295477867126, 0.1473011076450348, 0.00668632611